In [1]:
import pandas as pd
import numpy as np
import nltk

In [27]:
import string

In [93]:
lyrics_df=pd.read_csv("lyrics.csv",)

In [94]:
lyrics_df.head()

,Unnamed: 0,Song,Artist,Lyrics
0,0,18,One Direction,"I got a heart and I got a soul\nBelieve me, I ..."
1,1,2009,Mac Miller,Yeah\nEric\nWell\nI don't need to lie no more\...
2,2,24K Magic,Bruno Mars,Tonight\nI just wanna take you higher\nThrow y...
3,3,2U,David Guetta,No limit in the sky that I won't fly for ya (F...
4,4,47 (Remix),Anuel AA,"Mera, dime Ñengo\nReal G4 Life hasta la muerte..."


In [95]:
lyrics_df.drop(columns=lyrics_df.columns[0],inplace=True)

In [96]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [97]:
stop_words=stopwords.words('english')

In [122]:
def lyrics_to_words(lyrics):
    #remove punctuation
    for char in string.punctuation:
        lyrics=lyrics.replace(char,'')
    #lower case
    lyrics=lyrics.lower()
    
    #tokenize
    lyrics=lyrics.replace("/n",' ').split()
    return lyrics

def stem_lyrics(words):
    stemmer=PorterStemmer()
    return [stemmer.stem(word) for word in words]

def remove_stopwords(words,stop_words):
    return [word for word in words if word not in stop_words]

def rejoin_lyrics(words):
    return ' '.join(words)

In [154]:
def text_preproc(lyrics, remove_stop_words=True):
    
    if type(lyrics)==str:
        words=lyrics_to_words(lyrics)
    
        words=stem_lyrics(words)
        if remove_stop_words:
            words=remove_stopwords(words,stop_words)
        rejoined_lyrics=rejoin_lyrics(words)
        return rejoined_lyrics

In [111]:
lyrics_df.Lyrics[420]

nan

In [155]:
lyrics_df['LyricsProcessed']=lyrics_df['Lyrics'].apply(lambda x: text_preproc(x,remove_stop_words=False))

In [124]:
lyrics_df.LyricsProcessed

0      got heart got soul believ use made start fals ...
1      yeah eric well dont need lie nowaday shine tak...
2      tonight wanna take higher throw hand sky let s...
3      limit sky wont fli ya ya amount tear eye wont ...
4      mera dime ñengo real g4 life hasta la muert ca...
                             ...                        
620    thought id end sean wasnt match wrote song ric...
621    im good im new shit say ye instead thought saw...
622    whi love whi love whi love dont know whi stead...
623    dont know im good ive learn lose cant afford t...
624    babi dont feel good six word never understood ...
Name: LyricsProcessed, Length: 625, dtype: object

In [118]:
from sklearn.feature_extraction.text import CountVectorizer

In [133]:
cv=CountVectorizer()

vec=cv.fit(lyrics_df.LyricsProcessed.dropna())

vectorized_lyrics=vec.transform(lyrics_df.LyricsProcessed.dropna())

In [152]:
def get_top_n_words(corpus,n):
    cv=CountVectorizer()
    vec=cv.fit(corpus)
    vectorized_corpus=cv.transform(corpus)
    count_words=vectorized_corpus.sum(axis=0)
    word_freq=[(word,count_words[0,idx]) for word, idx in vec.vocabulary_.items()]
    word_freq_sorted=sorted(word_freq,key=lambda x: x[1],reverse=True)
    return word_freq_sorted[:n]

In [156]:
get_top_n_words(lyrics_df.LyricsProcessed.dropna(),50)

[('you', 9046),
 ('the', 8202),
 ('it', 5686),
 ('me', 5038),
 ('my', 4547),
 ('to', 4489),
 ('and', 4443),
 ('im', 3484),
 ('that', 3339),
 ('in', 3264),
 ('your', 3106),
 ('yeah', 3044),
 ('on', 2742),
 ('like', 2384),
 ('dont', 2150),
 ('up', 1958),
 ('we', 1893),
 ('no', 1853),
 ('of', 1846),
 ('know', 1844),
 ('all', 1760),
 ('be', 1747),
 ('for', 1687),
 ('with', 1654),
 ('but', 1627),
 ('got', 1571),
 ('love', 1569),
 ('is', 1551),
 ('so', 1504),
 ('oh', 1461),
 ('thi', 1419),
 ('just', 1406),
 ('get', 1378),
 ('what', 1330),
 ('when', 1226),
 ('go', 1136),
 ('let', 1093),
 ('she', 1074),
 ('if', 1016),
 ('do', 1008),
 ('now', 997),
 ('bitch', 991),
 ('they', 985),
 ('babi', 983),
 ('nigga', 969),
 ('one', 942),
 ('can', 910),
 ('caus', 899),
 ('out', 892),
 ('wa', 861)]

In [172]:
def get_top_n_words_by_artist(dataframe,n,remove_stop_words=True):
    artists=pd.unique(dataframe.Artist)
    
    table=[]
    for artist in artists:
        lyrics_processed=dataframe.Lyrics[dataframe.Artist==artist].dropna().apply(lambda x: text_preproc(x,remove_stop_words=remove_stop_words))
        dic={}
        top_n_words=get_top_n_words(lyrics_processed,n)
        for i in range(n):
            dic['Artist']=artist
            dic['Num_Songs']=len(lyrics_processed)
            word_key_name='Rank_'+str(i+1)+'_Word'
            word_value=top_n_words[i][0]
            count_key_name='Rank_'+str(i+1)+'_Count'
            count_value=top_n_words[i][1]
            dic[word_key_name]=word_value
            dic[count_key_name]=count_value
        table.append(dic)
    return pd.DataFrame(table)

In [174]:
top_words_by_artist=get_top_n_words_by_artist(lyrics_df,5)

In [178]:
pd.set_option('display.max_rows', 100)

top_words_by_artist

,Artist,Num_Songs,Rank_1_Word,Rank_1_Count,Rank_2_Word,Rank_2_Count,Rank_3_Word,Rank_3_Count,Rank_4_Word,Rank_4_Count,Rank_5_Word,Rank_5_Count
0,One Direction,9,know,39,love,37,nobodi,37,dont,35,like,34
1,Mac Miller,9,yeah,149,dont,58,im,56,like,42,aint,35
2,Bruno Mars,6,yeah,64,like,44,got,27,girl,25,know,24
3,David Guetta,5,name,25,say,23,oh,20,keep,19,ill,18
4,Anuel AA,3,lo,108,yo,104,la,91,que,88,el,72
5,Arctic Monkeys,10,wanna,48,come,41,mine,22,im,20,like,20
6,Ariana Grande,8,yeah,113,im,87,got,48,thank,43,next,43
7,The Beatles,10,let,50,jude,43,come,40,hey,27,never,26
8,Coldplay,9,oh,37,sky,21,wa,21,high,20,feel,19
9,Quavo,1,yeah,66,thi,24,hey,15,fast,12,go,11
